- required packages
    - numpy 
- Class | MonoPhone 
    - function | __init__
    - assigns attributes
        - regarding feature
            - .num_dims
        - regarding HMM
            - .phones
            - .num_phones
            - .num_states
            - .trans
            - .elem_prob
            - .state_prob
            - .loglikelihood
            - .trans.accumulatros
            - regarding Viterbi algorithm 
                - .alpha
                - .beta
                - .score
                - .viterbi_score
                - .track
        - regarding GMM
            - .num_mixture
            - .pdf
            - .pdf_accumulatros
        - regarding calculation
            - LZERO
            - LSMALL
            - ZERO
            - MINVAR
            
    - function | make_ptoro
    - create HMM prototype, the initial HMM 
        - variables 
            - phone_list, num_states, prob_loop, num_dims
        - steps
            - assign
                - phones, num_phones, num_states, num_mixture
            - create 
                - pdf for each phone, state, and mixture
                - transition matrix for each phone and state
                
    - function | calc_gconst
    - calculate log of the g constant
        - variables
            - variance
        - steps
            - calculate g constant 
            - D * log(2* pi) + np.sum(np.log(variance))
            
    - function | save_hmm
    - save attribute values, gaussian distribution parameters, transition probability matrices probabilities
        - variables
            - self, filename
        - steps
            - create hmmjson dictionary
            - save num_phones, num_states, num_mixture, num_dims with keys
            - save nested dictionaries with following order: 
                - HMMS = [Phones1, Phones2, Phones3 ..] | Phones1 = {phone, HMM} | 
                    - HMM = [State1, State2, State3 ...] | State1 = {state, trans, GMM} |
                        - GMM = [Mixture1, Mixture2, Mixture3 ...] | Mixture1 = {mixture, weight, mu, variance, gConst|
            - json.dump hmmjson to filename
    - function | load_hmm
    - load attribute values, gaussian distribution parameters, transition probability matrices probabilities
        - variables
            - self, filename
        - steps
            - read json file from filename
            - load num_phones, num_states, num_mixture, num_dims from json file to self
            - load nested transition matrix and pdf parameter matrix

    - function | calc_pdf
    - function | calc_out_prob
    - function | calc_alpha
    - function | calc_beta
    - function | flat_init
    - initialize gmm parameters for pdf in each phone, state and mixture
        - variables
            - self, mean, var
        - steps
            - open pdf[p][s][m] using for loop
            - assign the mean, variance and gConst for each pdf
    - function | reset_accumulators
    - function | update_accumulators
    - function | update_parameters
    - function | train
    - function | mixup
    - function | viterbi_deoding 
    - function | back_track
    - function | recognize
    - function | phone_alignment
    - function | log_add

In [ ]:
class MonoPhoneHMM():
    def __init__(self):
        self.num_dims = 1 # the dimensions of a feature 
        
        self.num_mixture = 1 # the number of mixes of GMM
        self.pdf = None # parameters for Single Gaussian Model
        
        self.phones = [] # list of phones 
        self.num_phones = 1 # the number of phones
        self.num_states = 1 # the number of states for each phone
        self.trans = None # transition probability matrix
        
        self.elem_prob = None # log probability for each normal distribution 
        self.state_prob = None # log probability for each state 
        self.loglikelihood = 0 # HMM likelihood
        
        self.alpha = None # forward probability
        self.beta = None # backward probability
        
        self.pdf_accumulators = None # variable for pdf parameter update
        self.trans_accumulators = None # variable for trans parameter update
        self.viterbi_score = 0 # score calculated by viterbi algorithm
        self.score = None # accumulated probability for viterbi algorithm
        self.track = None # viterbi track
        
        self.LZERO = -1E10 # aprroximate log(0) value
        self.LSMALL = -0.5E10 # the low boundary value from which to ignore
        self.ZERO = 1E-100 # approximate 0 
        self.MINVAR = 1E-4 # the value to add for variation flooring
        
    def make_proto(self, 
                   phone_list, 
                   num_states,
                   prob_loop,
                   num_dims):
        # assign phone_list, num_phones, num_states, num_dims, num_mixture
        self.phones = phone_list
        self.num_phones = len(self.phones)
        self.num_states = num_states 
        self.num_dims = num_dims
        self.num_mixture = 1
        
        # create pdf for each p, s, m
        self.pdf = []
        for p in range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                tmp_s = []
                for m in range(self.num_mixture):
                    mu = np.zeros(self.num_dims)
                    var = np.ones(self.num_dims)
                    weight = 1.0
                    gconst = self.calc_gconst(var)
                    gaussian = {'weight': weight, 
                               'mu': mu, 
                               'var': var, 
                               'gConst': gconst}
                    tmp_s.append(gaussian)
                tmp_p.append(tmp_s)
            self.pdf.append(tmp_p)
            
            
            
        # create log probability of transition
        prob_next = 1.0 - prob_loop
        log_prob_next = np.log(prob_next) if prob_next > self.ZERO else self.LZERO
        log_prob_loop = np.log(prob_loop) if prob_loop > self.ZERO else self.LZERO
        
        # create transition matrix for each p, s
        self.trans = []
        for p in range(self.num_phones):
            tmp_p = []
            for s in range(self.num_states):
                tmp_trans = np.array([log_prob_loop, log_prob_next])
                tmp_p.append(tmp_trans)
            self.trans.append(tmp_p)
            
            
    def calc_gconst(self, variance):
        # calculate gconst
        gconst = self.num_dims * np.log(2.0 * np.pi) + np.sum(np.log(variance))
        
        return gconst
    
    def flat_init(self, mean, var):
        # open pdf for each phone, state and mixture
        for p in range(self.num_phones):
            for s in range(self.num_states):
                for m in range(self.num_mixture):
                    pdf = self.pdf[p][s][m]
                    # assign mean, variance and gConst for each pdf
                    pdf['mu'] = mean
                    pdf['var'] = var
                    pdf['gConst'] = self.calc_gconst(var)
    
    def load_hmm(self, filename):
        # open hmm json file
        with open(filename, mode = 'r') as f:
            hmmjson = json.load(f)
            
        # read and load num_ attributes
        self.num_phones = hmmjson['num_phones']
        self.num_states = hmmjson['num_states']
        self.num_mixture = hmmjson['num_mixture']
        self.num_dims = hmmjson['num_dims']
        
        # read and load phones
        self.phones = []
        for p in range(self.num_phones):
            hmms = hmmjson['hmms'][p]
            phone = hmms['phone']
            phone_list.append(phone)
        self.phones = phone_list
        
        # read and load hmm parameters
        self.trans = []
        for p in range(self.num_phones):
            tmp_p = []
            hmms = hmmjson['hmms'][p]
            for s in range(self.num_states):
                hmm = hmms['hmm'][s]
                tmp_trans = np.array(hmm['trans'])
                tmp_trans /= np.sum(tmp_trans)
                for i in [0, 1]:
                    trans[i] = np.log(trans[i]) if trans[i] > self.ZERO else self.LZERO
                tmp_p.append(tmp_trans)
            self.trans.append(tmp_p)
                
                
        # read and load gmm parameters
        self.pdf = []
        for p in range(self.num_phones):
            tmp_p = []
            hmms = hmmjson['hmms'][p]
            for s in range(self.num_states):
                tmp_s = []
                hmm = hmms['hmm'][s]
                for m in range(self.num_mixture):
                    gmm = hmm['gmm'][m]
                    weight = gmm['weight']
                    mu = np.array(gmm['mean'])
                    var = np.array(gmm['variance'])
                    gconst = np.array(gmm['gConst'])
                    gaussian = {'weight': weight, 
                               'mu': mu, 
                               'var': var, 
                               'gConst': gconst}
                    tmp_s.append(gaussian)
                tmp_p.append(tmp_s)
            self.pdf.append(tmp_p)
    
    def save_hmm(self, filename):
    # create empty hmmjson file
    hmmjson = {}
    # save attribute values
    hmmjson['num_phones'] = self.num_phones
    hmmjson['num_states'] = self.num_states
    hmmjson['num_mixture'] = self.num_mixture
    hmmjson['num_dims'] = self.num_dims
    
    # save each phone in hmms
    hmmjson['hmms'] = []
    
    for p, phone in range(self.num_phones):
        model_p = {}
        model_p['phone'] = phone
        model_p['hmm'] = []
        # save state for each phone in hmm
        for s in range(self.num_states):
            model_s = {}
            model_s['state'] = s
            model_s['trans'] = list(np.exp(self.trans[p][s]))
            model_s['gmm'] = []
            # save mixture for each state in phone 
            for m in range(self.num_mixture):
                model_m = {}
                model_m['mixture'] = m
                model_m['weight'] = self.pdf[p][s][m]['weight']
                model_m['mu'] = list(self.pdf[p][s][m]['mu'])
                model_m['variance'] = list(self.pdf[p][s][m]['var'])
                model_m['gConst'] = self.pdf[p][s][m]['gConst']
                model_s['gmm'].append(model_m)
            model_p['hmm'].append(model_s)
        hmmjson['hmms'].append(model_s)
    
    # out as a file
    with open(filename, mode = 'w') as f:
        json.dump(hmmjson, f, indent = 4)
        